In [1]:
from models import UNet3D
from datasets import Brains
import numpy as np
import os

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.transforms as transforms
from torch.utils.data import Dataset

In [8]:
net = UNet3D(input_nc=1,output_nc=2)
net.load_state_dict(torch.load('../ckpt/1to8_05_05_3d_ce/model_250_epochs'))
net=net.cuda();

In [9]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [10]:
def postprocess(network_output, mask_path='../convexmask.npy'):
    
    mask = np.load(mask_path)
    
    postproc = torch.zeros_like(network_output)
    postproc[0] = torch.FloatTensor(mask)[None,...]
#     postproc[1] = torch.FloatTensor(mask)[None,...]
    
    return network_output*postproc

def dice_score(input, target):
    smooth = 1e-3
    iflat = input.reshape((input.size(0),-1))
    tflat = target.reshape((target.size(0),-1))
    intersection = (iflat * tflat).sum(1)
    
    return ((2. * intersection + smooth) / (iflat.sum(1) + tflat.sum(1) + smooth))

def test_sequential(image, model, size=64, stride=64, verbose=False):
    #image = float tensor from dataloader
    model.eval()
    
    out = torch.zeros_like(image)
    
    for i in range(image.shape[3]//size):
        for j in range(image.shape[4]//size):
            crop = image[:,:,:,i*size:(i + 1)*size,j*size:(j+1)*size]
            crop_predicted = model(Variable(crop).cuda())
            crop_predicted = crop_predicted.max(1, keepdim=True)[1]
            out[:,:,:,i*size:(i+1)*size,j*size: (j+1)*size] = crop_predicted.data
    
    return out

def validate_train(model, postprocess=None):
    data = Brains(crop=False, folder='../scanmasks-train/')
    batch_size = 1
    valit = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0)
    dices = np.array([])
    for X_batch, masks_batch in valit:
        preds = test_sequential(X_batch, model)
#         print(preds.shape)
        if postprocess:
            preds = postprocess(preds)
#             print(preds.shape)
        d = dice_score(preds, masks_batch.type(torch.FloatTensor))
        dices = np.hstack([dices, d.numpy()])
        print(np.mean(dices), len(dices))
        
    return dices


In [11]:
from utils import validate, postprocess

validate(net, postprocess=postprocess)
# validate(net)

01_228
01_356
01_481
01_73
01_453
01_230
01_479
01_32
01_373


0.5551753408379025

In [ ]:
# 0.48293770684136283

In [ ]:
d = validate_train(net, postprocess=postprocess)

In [ ]:
def validate(model):
    data = Brains(crop=False, folder='../scanmasks-val/')
    batch_size = 2
    valit = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0)
    dices = np.array([])
    for X_batch, masks_batch in valit:
        preds = test_sequential(X_batch, model)
        d = dice_score(preds, masks_batch.type(torch.FloatTensor))
        dices = np.hstack([dices, d.numpy()])
        print(d)
        
    return np.mean(dices)

        

In [ ]:
def dice_score(input, target):
    smooth = 1e-3
    iflat = input.reshape((input.size(0),-1))
    tflat = target.reshape((target.size(0),-1))
    intersection = (iflat * tflat).sum(1)
    
    return ((2. * intersection + smooth) / (iflat.sum(1) + tflat.sum(1) + smooth))

In [ ]:
def test_sequential(image, model, size=64, stride=64, verbose=False):
    #image = float tensor from dataloader
    model.eval()
    
    out = torch.zeros_like(image)
    
    for i in range(image.shape[3]//size):
        for j in range(image.shape[4]//size):
            crop = image[:,:,:,i*size:(i + 1)*size,j*size:(j+1)*size]
            crop_predicted = model(Variable(crop).cuda())
            crop_predicted = crop_predicted.max(1, keepdim=True)[1]
            out[:,:,:,i*size:(i+1)*size,j*size: (j+1)*size] = crop_predicted.data
    
    return out

In [ ]:
d = validate(net)
d

In [ ]:
d[0].numpy()

In [ ]:
plt.imshow(b[1, 0, 10])

In [ ]:
dice_score(t, b.type(torch.FloatTensor))

In [ ]:
im = a[0, 0, 10]
plt.imshow(im, cmap='gray')

In [ ]:
aa=a.view(a.size(0),7*8, a.size(1), a.size(2), 64,64)

In [ ]:
aa.shape

In [ ]:
aa[0,25,:,12,...][0].shape

In [ ]:
plt.imshow(aa[0,25,:,12,...][0].numpy(), cmap='gray')